In [31]:
import numpy as np

# data I/O
data = open('input.txt', 'r').read()  # harus berupa file teks biasa
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for i, ch in enumerate(chars)}

# hyperparameters
hidden_size = 100  # ukuran hidden layer
seq_length = 25    # panjang urutan untuk unroll
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size) * 0.01  # input to hidden
Whh = np.random.randn(hidden_size, hidden_size) * 0.01  # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size) * 0.01  # hidden to output
bh = np.zeros((hidden_size, 1))  # bias hidden
by = np.zeros((vocab_size, 1))  # bias output

def lossFun(inputs, targets, hprev):
    """
    inputs, targets: list of integers
    hprev: initial hidden state
    returns: loss, gradients, and last hidden state
    """
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0

    # forward pass
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size, 1))  # one-hot encoding
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t - 1]) + bh)
        ys[t] = np.dot(Why, hs[t]) + by
        e_y = np.exp(ys[t] - np.max(ys[t]))  # softmax stabil
        ps[t] = e_y / np.sum(e_y)
        loss += -np.log(ps[t][targets[t], 0])  # cross-entropy loss

    # backward pass
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1  # backprop from softmax
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext
        dhraw = (1 - hs[t] * hs[t]) * dh  # tanh'
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t - 1].T)
        dhnext = np.dot(Whh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam)  # clip gradients

    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs) - 1]

def sample(h, seed_ix, n):
    """
    sample a sequence of integers from the model 
    h: hidden state
    seed_ix: seed character index
    n: number of characters to sample
    """
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []
    for t in range(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        e_y = np.exp(y - np.max(y))
        p = e_y / np.sum(e_y)
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes

# training loop
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by)
smooth_loss = -np.log(1.0 / vocab_size) * seq_length  # initial loss

for n in range(1_000_000):  # misal 10.000 iterasi
    if p + seq_length + 1 >= len(data) or n == 0:
        hprev = np.zeros((hidden_size, 1))  # reset RNN memory
        p = 0
    inputs = [char_to_ix[ch] for ch in data[p:p + seq_length]]
    targets = [char_to_ix[ch] for ch in data[p + 1:p + seq_length + 1]]

    # sample output
    if n % 100 == 0:
        sample_ix = sample(hprev, inputs[0], 200)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print('----\n%s\n----' % (txt,))

    # forward and backward
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % 100 == 0:
        print('iter %d, loss: %f' % (n, smooth_loss))

    # parameter update with Adagrad
    for param, dparam, mem in zip(
        [Wxh, Whh, Why, bh, by],
        [dWxh, dWhh, dWhy, dbh, dby],
        [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8)

    p += seq_length
    


data has 67 characters, 20 unique.
----
tyjwgfkjj2tno2h2rnkjyamfuzfoiezg2yfyyieyemozuzsenmarhmh2hnfieejnkmyggfhstreh2gghwyaworsouafgntuerm2skgaaemzi2gaaj shuignwjoimm2aaanujg2ohufnjzzuztyht2jietyze2umnemw jw szkgo uy yjuwrgi mmejwyhesogyfak
----
iter 0, loss: 74.893315
----
ama neonsana ueuaa morawaniijuruaniijuruwknarekaminijska nnajuya anaza u kamjzoua   hraraniizuraiwn trorman narrm aiteknmanihfkuuanakzariaijzayas   2era2azanuraz eijuknwnn afkyaiittekyianijrraaniisfra
----
iter 100, loss: 73.059155
----
ama teknik a aormaf tfknmkzahrawani jurusan teonia  tsknama rurusan teknin anfya ka za az zahrawani  rawani jurusan teknik inforik informan jurus zahrawani teknik ijraufni tfowani hrrwami jurusanitfor
----
iter 200, loss: 67.344677
----
ama zaya kanza az zanjanzkanzayaz za rawani jurusan arrwani teawani jurusan teknik informan tekuik aza az aahrawani jurkani hrawani teknik informanaju azazahrawani jurusan tekn ka saya k nza az zahraw
----
iter 300, loss: 61.214290
----
ama saya kainf

In [33]:
def save_model_as_npz(Wxh, Whh, Why, bh, by, file_name='model.npz'):
    np.savez(file_name, Wxh=Wxh, Whh=Whh, Why=Why, bh=bh, by=by)
    print("Model saved as npz.")
    
save_model_as_npz(Wxh, Whh, Why, bh, by, file_name='model_100jt.npz')

Model saved as npz.


In [24]:
def load_model_from_npz(file_name='model.npz'):
    data = np.load(file_name)
    Wxh = data['Wxh']
    Whh = data['Whh']
    Why = data['Why']
    bh = data['bh']
    by = data['by']
    print("Model loaded from npz.")
    return Wxh, Whh, Why, bh, by

load_model_from_npz(file_name='model.npz')


Model loaded from npz.


(array([[-1.78544833e-02, -7.05797332e-01,  6.67746772e-01, ...,
          4.77228048e-01,  1.22631713e+00, -1.25721967e+00],
        [ 5.71186434e-01,  2.19606185e-01, -8.71056277e-01, ...,
          8.37029699e-01,  4.49410292e-01,  4.88577469e-02],
        [ 1.98349893e-02,  2.37954465e-01, -8.77312444e-02, ...,
          1.31632059e-01, -2.21236848e-01,  5.12000107e-01],
        ...,
        [ 2.18844544e-02,  3.45071975e-01, -1.88821104e-01, ...,
         -9.25221571e-04, -3.08945659e-01,  1.71639631e-01],
        [ 3.92771163e-01,  8.79590485e-01, -1.55858667e-01, ...,
          1.00984846e+00,  6.49161363e-01,  1.47053871e-01],
        [ 7.37251269e-02,  1.07509880e-01,  2.92469735e-01, ...,
         -1.71317445e-01, -2.08057567e-01,  2.50223706e-01]]),
 array([[-0.09277774, -0.04538019,  0.01469451, ...,  0.29024301,
          0.0688814 ,  0.04455232],
        [-0.16808196, -0.56481589, -0.11960895, ...,  0.29794943,
          0.01195799, -0.03100685],
        [ 0.10423288,  0.

In [25]:
# Setelah memuat model
hprev = np.zeros((hidden_size, 1))  # inisialisasi hidden state
seed_ix = char_to_ix['T']  # karakter pertama yang akan digunakan sebagai seed, misalnya 'T'
n = 100  # jumlah karakter yang ingin disampling

sample_ix = sample(hprev, seed_ix, n)  # hasil sampling
txt = ''.join(ix_to_char[ix] for ix in sample_ix)  # mengubah indeks kembali ke karakter
print('----\n%s\n----' % (txt,))


----
UVDEFGHIJKLMNOPQRSTUVWXYZQRSTUVWXYZNOPQRSTUVWXYZQRSTUVWXYZKLMNOPQRSTUVWXYZXYZKLMNOPQRSTUVWXYZOPQRSTU
----
